![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/1.4.Resume_MedicalNer_Model_Training.ipynb)

# 1.4 Resume MedicalNer Model Training

Steps:
- Train a new model for a few epochs.
- Load the same model and train for more epochs on the same taxnonomy, and check stats.
- Train a model already trained on a different data

## Colab Setup

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [4]:
from johnsnowlabs import nlp, medical, visual
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

👌 Detected license file /content/4.2.4.spark_nlp_for_healthcare (1).json
👌 Launched cpu optimized session with with: 🚀Spark-NLP==4.2.4, 💊Spark-Healthcare==4.2.4, running on ⚡ PySpark==3.1.2


## Download Clinical Word Embeddings for training

In [5]:
clinical_embeddings = nlp.WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]


## Download Data for Training (NCBI Disease Dataset)

In [6]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_test.conll
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NCBI_disease_official_train_dev.conll

In [8]:
# from sparknlp.training import CoNLL

training_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_train_dev.conll')

training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Identification of...|[{document, 0, 89...|[{document, 0, 89...|[{token, 0, 13, I...|[{pos, 0, 13, NN,...|[{named_entity, 0...|
|The adenomatous p...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
|Complex formation...|[{document, 0, 63...|[{document, 0, 63...|[{token, 0, 6, Co...|[{pos, 0, 6, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [9]:
test_data = nlp.CoNLL().readDataset(spark, 'NCBI_disease_official_test.conll')

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Clustering of mis...|[{document, 0, 10...|[{document, 0, 10...|[{token, 0, 9, Cl...|[{pos, 0, 9, NN, ...|[{named_entity, 0...|
|Ataxia - telangie...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 5, At...|[{pos, 0, 5, NN, ...|[{named_entity, 0...|
|The risk of cance...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 2, Th...|[{pos, 0, 2, NN, ...|[{named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



## Split the test data into two parts:
- We Keep the first part separate and use it for training the model further, as it will be totally unseen data from the same taxonomy.

- The second part will be used to testing and evaluating

In [10]:
(test_data_1, test_data_2) = test_data.randomSplit([0.5, 0.5], seed = 100)

# save the test data as parquet for easy testing
clinical_embeddings.transform(test_data_1).write.parquet('test_1.parquet')

clinical_embeddings.transform(test_data_2).write.parquet('test_2.parquet')

## Train a new model, pause, and resume training on the same dataset.

### Create a graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow (<= 2.7 ) is available. The graph is stored in the defined folder and loaded by the approach.

In [ ]:
!pip install -q tensorflow==2.7.0
!pip install -q tensorflow-addons

In [12]:
graph_folder_path = "medical_ner_graphs"

ner_graph_builder = medical.TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setLabelColumn("label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(24)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

In [13]:
# You can create custom TensorFlow graph file (`.pb` extension) for NER model training externally by using the script below. 
# If this method is used, graph folder should be added to NerApproach as  `.setGraphFolder(graph_folder_path)` .

'''
medical.tf_graph.print_model_params("ner_dl")

graph_folder_path = "medical_ner_graphs"

medical.tf_graph.build("ner_dl",
               build_params={"embeddings_dim": 200,
                             "nchars": 128,
                             "ntags": 12,
                             "is_medical": 1},
                model_location=graph_folder_path,
                model_filename="auto")
'''

'\nmedical.tf_graph.print_model_params("ner_dl")\n\ngraph_folder_path = "medical_ner_graphs"\n\nmedical.tf_graph.build("ner_dl",\n               build_params={"embeddings_dim": 200,\n                             "nchars": 128,\n                             "ntags": 12,\n                             "is_medical": 1},\n                model_location=graph_folder_path,\n                model_filename="auto")\n'

### Train for 2 epochs

In [14]:
nerTagger = medical.NerApproach()\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setLabelColumn("label")\
    .setOutputCol("ner")\
    .setMaxEpochs(2)\
    .setLr(0.003)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setTestDataset('./test_2.parquet')\
    .setGraphFolder(graph_folder_path)\
    .setOutputLogsPath('./ner_logs')

ner_pipeline = nlp.Pipeline(stages=[
    clinical_embeddings,
    ner_graph_builder,
    nerTagger
 ])

In [15]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 12.6 s, sys: 777 ms, total: 13.4 s
Wall time: 2min 18s


In [16]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_28b1fb173be1.log


In [17]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 49.12s - loss: 2501.0752 - avg training loss: 3.1460066 - batches: 795
Quality on test dataset: 
time to finish evaluation: 2.05s
Total test loss: 91.0160	Avg test loss: 1.3790
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 455	 41	 128	 0.9173387	 0.780446	 0.84337354
B-Disease	 428	 76	 90	 0.8492063	 0.82625484	 0.8375734
tp: 883 fp: 117 fn: 218 labels: 2
Macro-average	 prec: 0.8832725, rec: 0.80335045, f1: 0.8414179
Micro-average	 prec: 0.883, rec: 0.8019982, f1: 0.8405521


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 45.86s - loss: 1088.9698 - avg training loss: 1.3697734 - batches: 795
Quality on test dataset: 
time to finish evaluation: 1.22s
Total test loss: 63.7403	Avg test loss: 0.9658
label	 tp	 fp	 fn

In [18]:
# Logs of 4 consecutive epochs to compare with 2+2 epochs on separate datasets from same taxonomy

#!cat ner_logs/MedicalNerApproach_4d3d69967c3f.log

### Evaluate

In [19]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|424.0|83.0|91.0|515.0|   0.8363|0.8233|0.8297|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8297455968688845|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8297455968688845|
+------------------+

None


`ner_utils`: This new module is used after NER training to calculate mertic chunkbase and plot training logs.

`evaluate`: if verbose, returns overall performance, as well as performance per chunk type; otherwise, simply returns overall precision, recall, f1 scores

`loss_plot`: Plots the figure of loss vs epochs

`get_charts` : Plots the figures of metrics ( precision, recall, f1) vs epochs

```
from sparknlp_jsl.utils.ner_utils import get_charts, loss_plot, evaluate

metrics = evaluate(preds_df['prediction'].values, preds_df['ground_truth'].values)

loss_plot(f"./ner_logs/{log_file}")

get_charts('./ner_logs/'+log_files[0])
```

### Save the model to disk

In [20]:
ner_model.stages[2].write().overwrite().save('models/NCBI_NER_2_epoch')

### Train using the saved model on unseen dataset
#### We use unseen data from same taxonomy

In [21]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("models/NCBI_NER_2_epoch") ## load existing model
    
ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [22]:
# remove the existing logs

! rm -r ./ner_logs

In [23]:
%%time
ner_model_retrained = ner_pipeline.fit(test_data_1)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 79, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_79.pb
CPU times: user 8.31 s, sys: 198 ms, total: 8.5 s
Wall time: 25.4 s


In [24]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_2e42f9ef5b27.log


In [25]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 434


Epoch 1/2 started, lr: 0.003, dataset size: 434


Epoch 1/2 - 5.36s - loss: 76.32946 - avg training loss: 1.363026 - batches: 56
Quality on test dataset: 
time to finish evaluation: 1.86s
Total test loss: 60.7239	Avg test loss: 0.9201
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 502	 78	 81	 0.86551726	 0.8610635	 0.86328465
B-Disease	 448	 71	 70	 0.86319846	 0.8648649	 0.86403084
tp: 950 fp: 149 fn: 151 labels: 2
Macro-average	 prec: 0.8643578, rec: 0.86296415, f1: 0.8636604
Micro-average	 prec: 0.8644222, rec: 0.862852, f1: 0.8636364


Epoch 2/2 started, lr: 0.0029850747, dataset size: 434


Epoch 2/2 - 3.36s - loss: 62.552315 - avg training loss: 1.1170056 - batches: 56
Quality on test dataset: 
time to finish evaluation: 1.26s
Total test loss: 66.0241	Avg test loss: 1.0004
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 480	 75	 

In [26]:
#from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

pred_df = ner_model_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label",case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+----+-----+---------+------+------+
| entity|   tp|   fp|  fn|total|precision|recall|    f1|
+-------+-----+-----+----+-----+---------+------+------+
|Disease|440.0|116.0|75.0|515.0|   0.7914|0.8544|0.8217|
+-------+-----+-----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8216619981325863|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8216619981325863|
+------------------+

None


### Train a Model Choosing Best Model

`useBestModel`:  This param preserves and restores the model that has achieved the best performance at the end of the training. The priority is metrics from testDataset (micro F1), metrics from validationSplit (micro F1), and if none is set it will keep track of loss during the training. <br/>

`earlyStopping`: You can stop training at the point when the perforfmance on test/validation dataset starts to degrage. Two params are used in order to use this feature: <br/> 

- `earlyStoppingCriterion`: This is used set the minimal improvement of the test metric to terminate training. The metric monitored is the same as the metrics used in `useBestModel` (micro F1 when using test/validation set, loss otherwise). Default is 0 which means no early stopping is applied. 

- `earlyStoppingPatience`:  The number of epoch without improvement which will be tolerated. Default is 0, which means that early stopping will occur at the first time when performance in the current epoch is no better than in the previous epoch.

In [27]:
# remove the existing logs

! rm -r ./ner_logs

In [28]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(20)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('./test_2.parquet')\
      .setGraphFolder(graph_folder_path)\
      .setOutputLogsPath('./ner_logs')\
      .setValidationSplit(0.2)\
      .setUseBestModel(True)\
      .setEarlyStoppingCriterion(0.04)\
      .setEarlyStoppingPatience(3)


ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [29]:
%%time
ner_model = ner_pipeline.fit(training_data)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 10.1 s, sys: 308 ms, total: 10.4 s
Wall time: 3min 28s


In [30]:
!ls ner_logs/MedicalNerApproach*

ner_logs/MedicalNerApproach_227128638249.log


In [31]:
# Training Logs
! cat ner_logs/MedicalNerApproach*

Name of the selected graph: /content/medical_ner_graphs/blstm_3_200_24_85.pb
Training started - total epochs: 20 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 84 - training examples: 5093


Epoch 1/20 started, lr: 0.003, dataset size: 5093


Epoch 1/20 - 41.50s - loss: 2085.8096 - avg training loss: 3.2641778 - batches: 639
Quality on validation dataset (20.0%), validation examples = 1018
time to finish evaluation: 3.82s
Total validation loss: 191.4501	Avg validation loss: 1.2117
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 1064	 243	 251	 0.81407803	 0.8091255	 0.8115942
B-Disease	 912	 180	 220	 0.83516484	 0.8056537	 0.8201439
tp: 1976 fp: 423 fn: 471 labels: 2
Macro-average	 prec: 0.82462144, rec: 0.8073896, f1: 0.8159146
Micro-average	 prec: 0.8236765, rec: 0.80751944, f1: 0.8155179
Quality on test dataset: 
time to finish evaluation: 1.35s
Total test loss: 99.3184	Avg test loss: 1.5048
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 497	 120	 86	 0.8055105	 0.85248715	 0.8283333
B-

As you see above, training was stopped before 20th epoch since there were not improvement.

**Evaluate**

In [32]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_1))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|367.0|86.0|73.0|440.0|   0.8102|0.8341|0.8219|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8219484882418813|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8219484882418813|
+------------------+

None


In [33]:
pred_df = ner_model.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|444.0|81.0|71.0|515.0|   0.8457|0.8621|0.8538|
+-------+-----+----+----+-----+---------+------+------+

+------------------+
|             macro|
+------------------+
|0.8538461538461538|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.8538461538461538|
+------------------+

None


## Now let's take a model trained on a different dataset and train on this dataset

In [34]:
jsl_ner = medical.NerModel.pretrained('ner_jsl','en','clinical/models')

jsl_ner.getClasses()

ner_jsl download started this may take some time.
[OK!]


['O',
 'B-Injury_or_Poisoning',
 'B-Direction',
 'B-Test',
 'I-Route',
 'B-Admission_Discharge',
 'B-Death_Entity',
 'I-Oxygen_Therapy',
 'B-Relationship_Status',
 'I-Drug_BrandName',
 'B-Duration',
 'I-Alcohol',
 'I-Triglycerides',
 'I-Date',
 'B-Hyperlipidemia',
 'B-Respiration',
 'I-Test',
 'B-Birth_Entity',
 'I-VS_Finding',
 'B-Age',
 'I-Vaccine_Name',
 'I-Social_History_Header',
 'B-Labour_Delivery',
 'I-Medical_Device',
 'B-Family_History_Header',
 'B-BMI',
 'I-Fetus_NewBorn',
 'I-BMI',
 'B-Temperature',
 'I-Section_Header',
 'I-Communicable_Disease',
 'I-ImagingFindings',
 'I-Psychological_Condition',
 'I-Obesity',
 'I-Sexually_Active_or_Sexual_Orientation',
 'I-Modifier',
 'B-Alcohol',
 'I-Temperature',
 'I-Vaccine',
 'I-Symptom',
 'I-Pulse',
 'B-Kidney_Disease',
 'B-Oncological',
 'I-EKG_Findings',
 'B-Medical_History_Header',
 'I-Relationship_Status',
 'B-Cerebrovascular_Disease',
 'I-Blood_Pressure',
 'I-Diabetes',
 'B-Oxygen_Therapy',
 'B-O2_Saturation',
 'B-Psychological_C

### Now train a model using this model as base

In [35]:
nerTagger = medical.NerApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(2)\
      .setLr(0.003)\
      .setBatchSize(8)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setTestDataset('/content/test_2.parquet')\
      .setOutputLogsPath('ner_logs')\
      .setGraphFolder(graph_folder_path)\
      .setPretrainedModelPath("/root/cache_pretrained/ner_jsl_en_4.2.0_3.0_1666181370373")\
      .setOverrideExistingTags(True) # since the tags do not align, set this flag to true
    
# do hyperparameter by tuning the params above (max epoch, LR, dropout etc.) to get better results
ner_pipeline = nlp.Pipeline(stages=[
      clinical_embeddings,
      ner_graph_builder,
      nerTagger
 ])

In [36]:
# remove the existing logs

! rm -r ./ner_logs

In [37]:

%%time
ner_jsl_retrained = ner_pipeline.fit(training_data)


TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 3, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 24}
ner_dl graph exported to medical_ner_graphs/blstm_3_200_24_85.pb
CPU times: user 12.2 s, sys: 406 ms, total: 12.6 s
Wall time: 7min 49s


In [38]:
!cat ./ner_logs/MedicalNerApproach*

Name of the selected graph: pretrained graph
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 3 - chars: 103 - training examples: 6347


Epoch 1/2 started, lr: 0.003, dataset size: 6347


Epoch 1/2 - 216.00s - loss: 1320.7665 - avg training loss: 1.6613414 - batches: 795
Quality on test dataset: 
time to finish evaluation: 11.11s
Total test loss: 65.4080	Avg test loss: 0.9910
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Disease	 498	 58	 85	 0.89568347	 0.8542024	 0.8744513
B-Disease	 429	 72	 89	 0.8562874	 0.8281853	 0.8420019
tp: 927 fp: 130 fn: 174 labels: 2
Macro-average	 prec: 0.87598544, rec: 0.84119385, f1: 0.8582372
Micro-average	 prec: 0.8770104, rec: 0.84196186, f1: 0.8591289


Epoch 2/2 started, lr: 0.0029850747, dataset size: 6347


Epoch 2/2 - 207.35s - loss: 661.9785 - avg training loss: 0.83267736 - batches: 795
Quality on test dataset: 
time to finish evaluation: 10.09s
Total test loss: 54.7949	Avg test loss: 0.8302
label	 tp	 fp	 fn	 prec	 rec	 f1
I-Dise

In [39]:
#from sparknlp_jsl.eval import NerDLMetrics

pred_df = ner_jsl_retrained.stages[2].transform(clinical_embeddings.transform(test_data_2))

evaler = medical.NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label", drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+----+-----+---------+------+------+
| entity|   tp|  fp|  fn|total|precision|recall|    f1|
+-------+-----+----+----+-----+---------+------+------+
|Disease|440.0|98.0|75.0|515.0|   0.8178|0.8544|0.8357|
+-------+-----+----+----+-----+---------+------+------+

+-----------------+
|            macro|
+-----------------+
|0.835707502374169|
+-----------------+

None
+-----------------+
|            micro|
+-----------------+
|0.835707502374169|
+-----------------+

None
